<a href="https://colab.research.google.com/github/cy643/generative_ai/blob/main/0415_hw8/0415_hw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### HW8 AI Agents：職涯成長導師 + 超會寫 LinkedIn 的貼文助理

**原始任務：** 輸入在職場上遇到的低潮，生成一段具啟發性的LinkIn貼文。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種『為什麼這是一次成長契機』的理由
2. **第二階段（產文階段）**：從這五個理由中挑出最有啟發性的一個，寫成LinkIn貼文



#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Mistral】
api_key = userdata.get('Mistral')
os.environ['MISTRAL_API_KEY']=api_key
provider = "mistral"
model = "ministral-8b-latest"
print(api_key)

#【使用 OpenAI】
# api_key = userdata.get('OpenAI')
# os.environ['OPENAI_API_KEY']=api_key
# provider = "openai"
# model = "gpt-4o"

#【使用 Groq】
# api_key = userdata.get('Groq')
# os.environ['GROQ_API_KEY']=api_key
# provider = "groq"
# model = "llama3-70b-8192"

9y9H6qRaJTsj9UhFJauaccH9pjjqzCyJ


In [3]:
# Groq
# !pip install groq

# OpenAI
# !pip install openai

# Mistral
!pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.5/301.5 kB 19.8 MB/s eta 0:00:00


In [4]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 83.7 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mistralai 1.7.0 requires httpx>=0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.11.0 requires htt

### 2. 使用 AISuite 的準備

In [5]:
import aisuite as ai

In [6]:
provider_planner = "mistral"#"groq"
model_planner= "ministral-8b-latest"#"llama3-70b-8192"

provider_writer = "mistral"
model_writer = "ministral-8b-latest"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [7]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="mistral",
          model="ministral-8b-latest"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [8]:
system_planner = "你是一位資深職涯教練，擅長幫助人們從職涯低潮中看到成長機會。請針對使用者分享的挫折，思考五種『為什麼這是一次成長契機』的理由，列出條列式清單。請用台灣習慣的中文回應，可以用第一人稱思考。"
system_writer = "你是一位專業的 LinkedIn 小編，擅長用積極、真誠、有啟發性的語氣撰寫職涯貼文。請根據一個特別有啟發性的理由，寫出一段 LinkedIn 風格的第一人稱貼文。開頭要有一點個人故事，中間加入一個反思點，結尾呼應成長，並用 #成長思維 #職涯路上 等 Hashtag。"

In [9]:
def lucky_post(prompt):
    # Step 1: CoT - 思考五種幸運理由
    planning_prompt = f"使用者說：{prompt}。請幫我想五種這是一次成長契機的理由。"
    lucky_reasons = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    # Step 2: 選出最有趣一項，寫成貼文
    generation_prompt = f"這是我想到的五個理由：\n{lucky_reasons}\n\n請從中選出一個最有啟發性的理由，然後根據它寫一段 LinkedIn 職涯風格的貼文。"
    lucky_post = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return lucky_reasons, lucky_post

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [10]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00


In [11]:
import gradio as gr

In [12]:
with gr.Blocks() as demo:
    gr.Markdown("### 🚀 職涯低潮逆轉器 🌟")
    gr.Markdown("輸入一個你最近遇到的職涯挫折或難關，讓 AI 幫你找到其中的成長機會！")
    user_input = gr.Textbox(label="最近遇到的挑戰是…")
    btn = gr.Button("生成 LinkedIn 成長貼文 ✨")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 五個成長契機（Planning CoT）")
        out2 = gr.Textbox(label="📣 最終 LinkedIn 貼文（Po 文）")

    btn.click(lucky_post, inputs=[user_input], outputs=[out1, out2])

In [13]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://601ec03795f3b89466.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://601ec03795f3b89466.gradio.live
